# 🚀 OpenSloth Demo Training Notebook

This notebook have 2 main sections:
 1. Training a model with OpenSloth
 2. Training with unsloth

- Setup:
- Both use the same setup with datasets, sequences and global batch size.

In [1]:
# import wandb
# wandb.init(project="compare-unsloth")

In [ ]:
import os
os.environ['WANDB_PROJECT'] = 'compare-unsloth'
from opensloth.scripts.opensloth_sft_trainer import run_mp_training, setup_envs
from opensloth.opensloth_config import (
    OpenSlothConfig,
    HFDatasetConfig,
    FastModelArgs,
    LoraArgs,
    TrainingArguments,
)
from loguru import logger

# from transformers.training_args import TrainingArguments


# # Main configuration using Pydantic models
def get_configs(devices) -> tuple[OpenSlothConfig, TrainingArguments]:
    num_gpu = len(devices)
    opensloth_config = OpenSlothConfig(
        data=HFDatasetConfig(
            tokenizer_name="Qwen/Qwen3-8B",
            chat_template="qwen3",
            instruction_part="<|im_start|>user\n",
            response_part="<|im_start|>assistant\n",
            num_samples=10000,
            nproc=52,
            max_seq_length=4096,
            source_type="hf",
            dataset_name="mlabonne/FineTome-100k",
            split="train",
        ),
        devices=devices,  # list of int representing GPU ids
        fast_model_args=FastModelArgs(
            model_name="model_store/unsloth/Qwen3-14B-bnb-4bit",
            max_seq_length=4096,
            load_in_4bit=True,
        ),
        lora_args=LoraArgs(
            r=8,
            lora_alpha=16,
            target_modules=[
                "q_proj",
                "k_proj",
                "v_proj",
                "o_proj",
                "gate_proj",
                "up_proj",
                "down_proj",
            ],
            lora_dropout=0,
            bias="none",
            use_rslora=False,
        ),
        sequence_packing=True,
    )

    # # Training arguments using Pydantic model
    training_config = TrainingArguments(
        output_dir=f"outputs/exps/qwen3-14b-FineTome-{num_gpu}gpus-seql-packing",
        per_device_train_batch_size=1,
        gradient_accumulation_steps=64,  # Adjust based on n_gpu
        learning_rate=1e-5,
        logging_steps=1,
        num_train_epochs=1,
        lr_scheduler_type="linear",
        warmup_steps=5,
        save_total_limit=1,
        weight_decay=0.01,
        optim="adamw_8bit",
        seed=3407,
        report_to="wandb",  # tensorboard or wawndb
    )
    setup_envs(opensloth_config, training_config)
    return opensloth_config, training_config


if __name__ == "__main__":
    opensloth_config, training_config = get_configs(devices=[0])
    # opensloth_config, training_config = get_configs(devices=[0,1,2,3])
    run_mp_training(opensloth_config.devices, opensloth_config, training_config)

Global batch size: 64
[MP] Running on 1 GPUs


03:29:22 | INFO     | GPU0 | opensloth_sft_trainer.py:41 | Training on GPU 0 with output_dir outputs/exps/qwen3-14b-FineTome-1gpus-seql-packing
03:29:22 | INFO     | GPU0 | opensloth_sft_trainer.py:44 | 🚀 Starting total training timer


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Using compiler location: .cache/unsloth_compiled_cache_0
==((====))==  Unsloth 2025.5.9: Fast Qwen3 patching. Transformers: 4.52.4.
   \\   /|    NVIDIA H100 80GB HBM3. Num GPUs = 1. Max memory: 79.189 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 9.0. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.08it/s]
03:29:42 | INFO     | GPU0 | logging_config.py:161 | ⏱️  model_loading: 8.57s
03:29:42 | INFO     | GPU0 | nccl_grad_sync.py:124 | [GPU=0] NCCL env: RANK=0, WORLD_SIZE=1, MASTER_ADDR=127.0.0.1, MASTER_PORT=29501
03:29:42 | INFO     | GPU0 | nccl_grad_sync.py:128 | [GPU=0] Setting current CUDA device to:0, os.environ['CUDA_VISIBLE_DEVICES']='0'
03:29:42 | INFO     | GPU0 | init_modules.py:50 | Model loaded on device cuda:0, tokenizer: Qwen2TokenizerFast
03:29:45 | INFO     | GPU0 | logging_config.py:161 | ⏱️  lora_setup: 3.23s
03:29:45 | INFO     | GPU0 | init_modules.py:74 | Applied chat template: qwen3
03:29:45 | INFO     | GPU0 | logging_config.py:161 | ⏱️  model_init: 11.81s
03:29:45 | INFO     | GPU0 | init_modules.py:148 | Creating final SFTTrainer with prepared dataset...
03:29:46 | INFO     | GPU0 | init_modules.py:161 | Replacing DataCollatorForLanguageModeling with DataCollatorForSeq2Seq for better sequen

Unsloth: Making `model.base_model.model.model` require gradients
[LOCAL_RANK=0] Patching log. Dir: outputs/exps/qwen3-14b-FineTome-1gpus-seql-packing, GPUs: 1
[LOCAL_RANK=0] Log patch initialization complete.
🔧 Patching Trainer to use RandomSamplerSeededByEpoch


...

: 

: 

: 

: 

: 

## Unsloth default 

In [ ]:
import os

from opensloth.patching.patch_sampler import patch_sampler

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["OPENSLOTH_LOCAL_RANK"] = "0"


def train_qwen3_model():
    """Train Qwen3 model with minimal setup."""
    from opensloth.dataset_utils import get_tokenized_dataset, HFDatasetConfig

    text_dataset = get_tokenized_dataset(
        HFDatasetConfig(
            tokenizer_name="Qwen/Qwen3-8B",
            chat_template="qwen3",
            instruction_part="<|im_start|>user\n",
            response_part="<|im_start|>assistant\n",
            num_samples=10000,
            nproc=52,
            max_seq_length=4096,
            source_type="hf",
            dataset_name="mlabonne/FineTome-100k",
            split="train",
        ),
        do_tokenize=False,
    )
    from unsloth import FastLanguageModel
    import torch
    from trl import SFTTrainer, SFTConfig

    # Load model
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name="unsloth/Qwen3-0.6B-bnb-4bit",
        max_seq_length=4096,
        load_in_4bit=True,
        load_in_8bit=False,
        full_finetuning=False,
    )

    # Add LoRA adapters
    model = FastLanguageModel.get_peft_model(
        model,
        r=8,
        lora_alpha=16,
        target_modules=[
            "q_proj",
            "k_proj",
            "v_proj",
            "o_proj",
            "gate_proj",
            "up_proj",
            "down_proj",
        ],
        lora_dropout=0,
        bias="none",
        use_gradient_checkpointing=True,
        random_state=3407,
        use_rslora=False,
        loftq_config=None,
    )
    args = SFTConfig(
        output_dir="outputs/exps/qwen3-14b-FineTome-unsloth",
        dataset_text_field="text",
        per_device_train_batch_size=4,
        gradient_accumulation_steps=8*2, # Adjust based on n_gpu
        warmup_steps=5,
        learning_rate=1e-5,
        num_train_epochs=1,
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        report_to="wandb",  
    )

    # args.skip_prepare_dataset = True
    trainer = SFTTrainer(
        model=model,
        tokenizer=tokenizer,
        train_dataset=text_dataset,
        eval_dataset=None,
        args=args,
    )
    from unsloth_zoo.dataset_utils import train_on_responses_only

    trainer = train_on_responses_only(
        trainer,
        tokenizer=tokenizer,
        instruction_part="<|im_start|>user\n",
        response_part="<|im_start|>assistant\n",
    )

    # Show memory stats
    gpu_stats = torch.cuda.get_device_properties(0)
    start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
    max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
    print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
    print(f"{start_gpu_memory} GB of memory reserved.")

    # Train the model

    # from ._patch_sampler import patch_sampler

    trainer = patch_sampler(trainer)
    trainer_stats = trainer.train()

    # Show final memory and time stats
    used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
    used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
    used_percentage = round(used_memory / max_memory * 100, 3)
    lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
    print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
    print(
        f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
    )
    print(f"Peak reserved memory = {used_memory} GB.")
    print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
    print(f"Peak reserved memory % of max memory = {used_percentage} %.")
    print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

    return model, tokenizer


if __name__ == "__main__":
    model, tokenizer = train_qwen3_model()
    print("Training completed successfully!")

16:47:41 | INFO     | GPU0 | dataset_utils.py:222 | Preparing dataset 7fe3c373565b53a9...


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


Saving the dataset (0/1 shards):   0%|          | 0/10000 [00:00<?, ? examples/s]

==((====))==  Unsloth 2025.5.9: Fast Qwen3 patching. Transformers: 4.52.4.
   \\   /|    NVIDIA H100 80GB HBM3. Num GPUs = 1. Max memory: 79.189 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 9.0. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.5.9 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


...

16:49:02 | INFO     | GPU0 | patch_sampler.py:52 | 🎲 Sampler epoch 0: emitting 10000 indices
First ids dataset samples: [3771, 6672, 7261, 760, 3779, 1772, 7509, 2679, 2305, 9215]
...Last ids: [9674, 1424, 8935, 1679, 2286, 3657, 4012, 4506, 409, 1824]


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.334000
2,1.317700
3,1.291900
4,1.326100
5,1.373800
6,1.330500
7,1.337200
8,1.147600
9,1.200500
10,1.353700


17:13:00 | INFO     | GPU0 | patch_sampler.py:61 | 🎲 Sampler epoch 0: dataset_size=10000
   📋 First 10 indices: [3771, 6672, 7261, 760, 3779, 1772, 7509, 2679, 2305, 9215]
   📋 Last 10 indices: [9674, 1424, 8935, 1679, 2286, 3657, 4012, 4506, 409, 1824]


1444.214 seconds used for training.
24.07 minutes used for training.
Peak reserved memory = 1.725 GB.
Peak reserved memory for training = 0.913 GB.
Peak reserved memory % of max memory = 2.178 %.
Peak reserved memory for training % of max memory = 1.153 %.
Training completed successfully!
